# SVM com três kernels (Linear, Polinomial, RBF) — SVC (C) e NuSVC (nu)

Este notebook demonstra **SVM** em problemas multi-classes com três kernels diferentes:
- `linear`
- `poly` (polinomial)
- `rbf` (Radial Basis Function)

Para **cada kernel**, treinamos e avaliamos:
- **SVC**, controlado por **C**
- **NuSVC**, controlado por **nu**

Inclui:
- Geração de dataset sintético multi-classe
- Padronização (`StandardScaler`)
- Métricas (acurácia, matriz de confusão)
- Visualização da **fronteira de decisão** (apenas nas 2 primeiras features padronizadas, por visualização)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, NuSVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# 1) Dataset
X, y = make_classification(
    n_samples=1000,
    n_features=8,
    n_informative=4,
    n_redundant=2,
    n_classes=8,
    n_clusters_per_class=1,
    class_sep=1.2,
    random_state=37
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=37, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

def fit_and_eval_svc(kernel: str, degree: int = 3, C: float = 1.0, gamma: str | float = "scale"):
    if kernel == "poly":
        clf = SVC(kernel=kernel, degree=degree, C=C, gamma=gamma, decision_function_shape="ovr", random_state=42)
    else:
        clf = SVC(kernel=kernel, C=C, gamma=gamma, decision_function_shape="ovr", random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return {"model": clf, "y_pred": y_pred, "acc": acc, "cm": cm}

def fit_and_eval_nusvc(kernel: str, degree: int = 3, nu: float = 0.5, gamma: str | float = "scale"):
    if kernel == "poly":
        clf = NuSVC(kernel=kernel, degree=degree, nu=nu, gamma=gamma, decision_function_shape="ovr", random_state=42)
    else:
        clf = NuSVC(kernel=kernel, nu=nu, gamma=gamma, decision_function_shape="ovr", random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return {"model": clf, "y_pred": y_pred, "acc": acc, "cm": cm}

def plot_confusion(cm, title: str):
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(values_format='d')
    plt.title(title)
    plt.tight_layout()
    plt.show()

def plot_decision_boundary_2d(model, X_scaled, y, title: str):
    X_vis = X_scaled[:, :2]
    h = 0.05
    x_min, x_max = X_vis[:, 0].min() - 1, X_vis[:, 0].max() + 1
    y_min, y_max = X_vis[:, 1].min() - 1, X_vis[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    X_grid = np.c_[xx.ravel(), yy.ravel()]
    X_full = np.zeros((X_grid.shape[0], X_scaled.shape[1]))
    X_full[:, :2] = X_grid
    Z = model.predict(X_full)
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(6, 5))
    plt.contourf(xx, yy, Z, alpha=0.25)
    plt.scatter(X_vis[:, 0], X_vis[:, 1], c=y, s=18)
    plt.title(title)
    plt.xlabel("Feature 1 (scaled)")
    plt.ylabel("Feature 2 (scaled)")
    plt.grid(True, linewidth=0.3, alpha=0.5)
    plt.tight_layout()
    plt.show()

# Hiperparâmetros
C_lin, C_poly, C_rbf = 1.0, 1.0, 1.0
nu_lin, nu_poly, nu_rbf = 0.3, 0.3, 0.3
poly_degree = 3
gamma_rbf = "scale"

# Linear
svc_lin = fit_and_eval_svc(kernel="linear", C=C_lin)
nusvc_lin = fit_and_eval_nusvc(kernel="linear", nu=nu_lin)
print("=== Kernel Linear ===")
print(f"SVC (C={C_lin})  -> Acurácia: {svc_lin['acc']:.4f}")
print(f"NuSVC (nu={nu_lin}) -> Acurácia: {nusvc_lin['acc']:.4f}\n")
plot_confusion(svc_lin["cm"], "SVM Linear - SVC (C)")
plot_confusion(nusvc_lin["cm"], "SVM Linear - NuSVC (nu)")
plot_decision_boundary_2d(svc_lin["model"], X_test_scaled, y_test, "Fronteira - SVM Linear (SVC)")
plot_decision_boundary_2d(nusvc_lin["model"], X_test_scaled, y_test, "Fronteira - SVM Linear (NuSVC)")

# Polinomial
svc_poly = fit_and_eval_svc(kernel="poly", degree=poly_degree, C=C_poly, gamma="scale")
nusvc_poly = fit_and_eval_nusvc(kernel="poly", degree=poly_degree, nu=nu_poly, gamma="scale")
print("=== Kernel Polinomial ===")
print(f"SVC (degree={poly_degree}, C={C_poly})  -> Acurácia: {svc_poly['acc']:.4f}")
print(f"NuSVC (degree={poly_degree}, nu={nu_poly}) -> Acurácia: {nusvc_poly['acc']:.4f}\n")
plot_confusion(svc_poly["cm"], "SVM Polinomial - SVC (C)")
plot_confusion(nusvc_poly["cm"], "SVM Polinomial - NuSVC (nu)")
plot_decision_boundary_2d(svc_poly["model"], X_test_scaled, y_test, "Fronteira - SVM Polinomial (SVC)")
plot_decision_boundary_2d(nusvc_poly["model"], X_test_scaled, y_test, "Fronteira - SVM Polinomial (NuSVC)")

# RBF
svc_rbf = fit_and_eval_svc(kernel="rbf", C=C_rbf, gamma=gamma_rbf)
nusvc_rbf = fit_and_eval_nusvc(kernel="rbf", nu=nu_rbf, gamma=gamma_rbf)
print("=== Kernel RBF ===")
print(f"SVC (C={C_rbf}, gamma={gamma_rbf})  -> Acurácia: {svc_rbf['acc']:.4f}")
print(f"NuSVC (nu={nu_rbf}, gamma={gamma_rbf}) -> Acurácia: {nusvc_rbf['acc']:.4f}\n")
plot_confusion(svc_rbf["cm"], "SVM RBF - SVC (C)")
plot_confusion(nusvc_rbf["cm"], "SVM RBF - NuSVC (nu)")
plot_decision_boundary_2d(svc_rbf["model"], X_test_scaled, y_test, "Fronteira - SVM RBF (SVC)")
plot_decision_boundary_2d(nusvc_rbf["model"], X_test_scaled, y_test, "Fronteira - SVM RBF (NuSVC)")